---
title: Recommender systems, a celebration of collaborative filtering and content filtering
---

*Group 15 - Weidong Xu, Jiejun Lu*

**Note:** You can view source codes for this project on [GitHub](https://github.com/xuwd11/Recommender_Systems), including the python package [`recommender`](https://github.com/xuwd11/Recommender_Systems/tree/master/recommender) we wrote for this project.
<br>

## Contents
{:.no_toc}
*  
{: toc}

## Project Statement and Motivation

Recommender systems can predict the rating a user would give to an item by learning from the historical data of user's ratings, or the attributes of users and items if available. In this project, we constructed a recommender system for restaurants using an ensemble method, which combines the prediction of several base estimators, including baseline estimators, collaborative filtering estimators, and content filtering estimators. We benchmarked these base estimators, followed by exploring strategies of building the ensemble estimator. We demonstrated our recommender system performs robustly on different size of datasets.
<br><br>

## Introduction and Description of Data

### Dataset

In this project, we use "review" (shape: 4736897 X 9), "business" (shape: 156639 X 101)  and "user" (shape: 1183362 X 22) from [Yelp academic dataset](https://www.yelp.com/dataset/challenge). Each row in "review" specifies a review that a user makes on a restaurant (or other business premises such as barbershop), including date, comment ("text") and rating ("stars"), as well as the number of votes of different attributes received on this review. "Business" contains information on restaurants and barbershops appearing in "review", including typical attributes defining a restaurant and average ratings. There are a lot of missing values in "business", mostly caused by the missing of attribute descriptions. "User" contains information on users, including profile summaries, social networks on yelp and average ratings. "Review", "business" and "user" are linked together through "user\_id" and "business\_id". A peek of raw data is available [here](https://github.com/xuwd11/Recommender_Systems/blob/master/01_rawdata_peek.ipynb).

### Data Wrangling

To wrangle data for EDA and predictive modeling, we first checked and cleaned duplicate reviews (an user reviews a business for multiple times). We identified 1 case of duplicates involving 2 reviews; we simply dropped one of them since the ratings happen to be the same. Then we dropped business places unrelated to restaurants and closed restaurants (~16.4% of rows in "business"), and kept reviews and users associated with remaining restaurants. We conducted a series of EDA and found that ratings are correlated with many of the attributes in "business", "user" and "review", which inspired us to propose a content filtering model. We checked the number of restaurants in each city (there are 980 cities in the remaining dataset), and sampled several sets of different sizes by extracting data associated with restaurants in several cities of different sizes respectively for benchmarking (we chose Champaign, Cleveland, Pittsburgh, Toronto, and Las_Vegas). 

To build a recommender system, we can do collaborative filtering or content filtering. To perform collaborative filtering, we only need restaurant ratings from each user, which we can obtain by keeping 3 columns, i.e., "user\_id", "business\_id" and "stars", in "review". Content filtering requires a profile for each user or restaurant, which can characterize its nature; we can obtain the required data by merging "review" with "user" and "business" through "user\_id" and "business\_id" respectively.
<br><br>

## Literature Review/Related Work

There are primarily [2 strategies](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf) for recommender systems, content filtering and [collaborative filtering](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf). Content filtering requires a profile for each user or restaurant, which captures its nature, and makes predictions by learning from user or restaurant profiles. Collaborative filtering makes predictions by analyzing only past user behavior, such as restaurant ratings. Although collaborative filtering suffers from cold start problem due to its inability to handle new users and restaurants, collaborative filtering is extremely helpful when user or restaurant profile data required for content filtering are not available, and is generally more accurate than content filtering methods.

Collaborative filtering approaches include neighborhood methods and latent factor models. As demonstrated by the [Netflix Prize competition](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf), matrix factorization based latent factor models generally performs better than neighborhood methods.

We implemented some baseline models and latent factor models from scratch (by using `numpy` and `scipy`'s linear algebra toolkits instead of well-established recommender system packages); we implemented other algorithms by wrapping around methods in a recommender system python package, [`scikit-surprise`](http://surpriselib.com/). Each algorithm we implemented by using the [`scikit-surprise`](http://surpriselib.com/) package is indicated by a * after its name.

Besides reporting root mean square error (RMSE) and $R^2$ score, we found it would be very helpful to visualize model's performance on different ratings by rounding the predicted ratings to integers (and setting ratings below 1 to 1 and above 5 to 5), and plotting the confusion matrix in a format used in [a related work](https://github.com/kevin11h/YelpDatasetChallengeDataScienceAndMachineLearningUCSD/blob/master/Yelp%20Predictive%20Analytics.ipynb). 

A comprehensive list of references can be found in [References](09_reference.html).
<br><br>

## Result summary

We benchmarked the base estimators and ensemble estimators in 6 datasets of different sizes. In each dataset, we randomly split the reviews into 3 sets: a training set (60%), a cross-validation set (16%) and a test set (24%). We train base estimators on the training set, and test on the test set; cross-validation set is used for the training of ensemble estimators. All experiments are run on a desktop with Inter Xeon CPU 3.10 GHz, 256 GB RAM.

Please see [EDA](03_EDA.html), [Collaborative Filtering](04_collaborative_filtering.html) and [Content Filtering](05_content_filtering.html) for modeling approaches and project trajectory.

Please see [Results](07_results.html) for detailed results including confusion matrix visualization (of rounded predictions) for each experiment we list below.

Please see [Conclusions](08_conclusions.html) for conclusions and future work.
<br><br>

In [1]:
import sys
import traceback
import pandas as pd
import numpy as np
import time
from copy import deepcopy

from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from scipy import sparse

import pickle

from IPython.display import display, HTML, Markdown

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
pd.set_option('display.width', 15000)
pd.set_option('display.max_columns', 100)
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context('poster')
%matplotlib inline

from surprise import Dataset, Reader
from surprise import NormalPredictor, BaselineOnly, SVD, SVDpp, NMF, \
SlopeOne, CoClustering, KNNBasic, KNNWithMeans, KNNBaseline

from recommender import plot_cm, get_results, show_results, IO, \
show_summaries, get_base_predictions, get_multi_base_predictions
from recommender import ModeClassifier, BaselineMean, BaselineRegression, ALS1, ALS2, RS_surprise, RS_ensemble

In [2]:
cities = ['Champaign', 'Cleveland', 'Pittsburgh', 'Toronto', 'Las_Vegas', 'Full']

for city in cities:
    data_dir = 'data/{}/'.format(city)
    model_names = IO(data_dir + 'results/model_names.pkl').read_pickle()
    results = IO(data_dir + 'results/results.pkl').read_pickle()
    is_successful = IO(data_dir + 'results/is_successful.pkl').read_pickle()
    sizes = IO(data_dir + 'sizes.pkl').read_pickle()
    
    display(Markdown('### {} <sup>({} reviews, {} restaurants, {} users)</sup>'.\
                 format(city, sizes[0], sizes[1], sizes[2])))
    # display(Markdown('**Collaborative filtering**'))
    show_summaries(model_names, results, is_successful, title='Collaborative filtering')
    display(Markdown('<sup>(* shows the algorithms we implemented by wrapping around \
    methods in scikit-surprise python package)</sup>'))
    
    model_names = IO(data_dir + 'results05/model_names.pkl').read_pickle()
    results = IO(data_dir + 'results05/results.pkl').read_pickle()
    is_successful = IO(data_dir + 'results05/is_successful.pkl').read_pickle()
    #sizes = IO(data_dir + 'sizes.pkl').read_pickle()

    #display(Markdown('### {} <sup>({} reviews, {} restaurants, {} users)</sup>'.\
                     #format(city, sizes[0], sizes[1], sizes[2])))
    #display(Markdown('**Content filtering**'))
    show_summaries(model_names, results, is_successful, title='Content filtering')
    
    model_names = IO(data_dir + 'results06/model_names.pkl').read_pickle()
    results = IO(data_dir + 'results06/results.pkl').read_pickle()
    is_successful = IO(data_dir + 'results06/is_successful.pkl').read_pickle()
    #sizes = IO(data_dir + 'sizes.pkl').read_pickle()

    #display(Markdown('### {} <sup>({} reviews, {} restaurants, {} users)</sup>'.\
                     #format(city, sizes[0], sizes[1], sizes[2])))
    #display(Markdown('**Ensemble estimators**'))
    show_summaries(model_names, results, is_successful, title='Ensemble estimators')
    display(Markdown('<sup>(Ensemble1 represents the ensemble of collaborative filtering models; \
    Ensemble2 represents the ensemble of collaborative filtering and content filtering models)</sup>'))
        
    display(Markdown('''<br><br>'''))

### Champaign <sup>(20571 reviews, 878 restaurants, 8451 users)</sup>

Collaborative filtering,fitting time (s),train RMSE,test RMSE,train $R^2$,test $R^2$
Mode estimator,0.0000,1.9995,2.0258,-0.9501,-0.9500
Normal predictor*,0.0870,1.8825,1.8821,-0.7286,-0.6833
Baseline (mean),0.0190,0.9485,1.4648,0.5612,-0.0195
Baseline (regression),0.0350,1.0481,1.3032,0.4642,0.1930
Baseline (ALS)*,0.0570,1.1981,1.3200,0.2998,0.1721
KNN (basic)*,0.9841,0.4328,1.4642,0.9086,-0.0187
KNN (with means)*,1.2851,0.5898,1.5310,0.8303,-0.1138
KNN (baseline)*,1.0201,0.4175,1.3718,0.9150,0.1058
SVD-ALS1,12.2077,0.6747,1.3064,0.7780,0.1891
SVD-ALS2,12.9087,0.6764,1.3092,0.7768,0.1855


<sup>(* shows the algorithms we implemented by wrapping around     methods in scikit-surprise python package)</sup>

Content filtering,fitting time (s),train RMSE,test RMSE,train $R^2$,test $R^2$
Ridge regression,0.0690,1.0773,1.0971,0.4339,0.4280
Random forest,1.0951,1.0262,1.0862,0.4864,0.4394


Ensemble estimators,fitting time (s),train RMSE,test RMSE,train $R^2$,test $R^2$
Ensemble1 (weighted average),0.000,0.8527,1.3071,0.6454,0.1882
Ensemble1 (Ridge regression),0.011,1.3268,1.3026,0.1413,0.1937
Ensemble1 (random forest),0.220,1.0506,1.3048,0.4617,0.1910
Ensemble2 (weighted average),0.000,0.9007,1.1591,0.6043,0.3616
Ensemble2 (Ridge regression),0.004,1.2721,1.0830,0.2107,0.4426
Ensemble2 (random forest),0.271,1.0678,1.0847,0.4439,0.4409


...

<sup>(* shows the algorithms we implemented by wrapping around     methods in scikit-surprise python package)</sup>

Content filtering,fitting time (s),train RMSE,test RMSE,train $R^2$,test $R^2$
Ridge regression,17.1610,1.0857,1.0869,0.4171,0.4167
Random forest,663.2849,1.0639,1.0653,0.4403,0.4396


Ensemble estimators,fitting time (s),train RMSE,test RMSE,train $R^2$,test $R^2$
Ensemble1 (weighted average),0.0000,0.8161,1.2572,0.6706,0.2195
Ensemble1 (Ridge regression),0.4160,1.0550,1.2520,0.4496,0.2260
Ensemble1 (random forest),56.0282,1.0652,1.2542,0.4389,0.2233
Ensemble2 (weighted average),0.0000,0.8864,1.1434,0.6115,0.3545
Ensemble2 (Ridge regression),0.5530,1.0865,1.0623,0.4163,0.4427
Ensemble2 (random forest),69.6950,1.0689,1.0703,0.4351,0.4343


<sup>(Ensemble1 represents the ensemble of collaborative filtering models;     Ensemble2 represents the ensemble of collaborative filtering and content filtering models)</sup>

<br><br>

## Authors

* [Weidong Xu](https://github.com/xuwd11)

* [Jiejun Lu](https://github.com/gwungwun)